<a href="https://colab.research.google.com/github/ryanhao1115/ML-for-Fraud-Detection/blob/main/5_3_DL_onehot_SMOTE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install imbalanced-learn

## Deep Learning model with One hot encoding data
1. import new return label data.
2. Under sampling data to address imbalance class issue.
4. Train model
5. Eval model
6. Create Risk-heatmap

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.backend import clear_session
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
import tensorflow as tf
import random
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline


/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
def reset_seeds():
    '''
    reset random seeds for modeling
    '''
    np.random.seed(1)
    random.seed(2)
    if tf.__version__[0] == '2':
        tf.random.set_seed(3)
    else:
        tf.set_random_seed(3)
    print("RANDOM SEEDS RESET")

In [ ]:
## import dataset 
path = '/content/drive/MyDrive/Colab Notebooks/finalproject/'
file = path + 'sales_clean.csv'
df = pd.read_csv(file)

In [ ]:
df = df.drop(columns='Unnamed: 0')

In [ ]:
df.nunique()

distributor        639
sales              209
branch              28
inv_type             3
invoice_no       30721
product_no        1094
prod_cla             7
qty                202
total_amt        16112
sale_price        8598
ship_qty           202
cust_type           20
return               2
discount_app         2
list_price        6720
inv_ship_days      133
dtype: int64

## import return label data

In [ ]:
## import fraud data
file = path + 'frauds.csv'
df_fraud = pd.read_csv(file,header=None)
df_fraud.columns = ['invoice_no']

In [ ]:
## import return new labeled data
file = path + 'return.csv'
df_fraud2 = pd.read_csv(file,header=None)
df_fraud2.columns = ['invoice_no']

In [ ]:
len(df_fraud2)

24

In [ ]:
def label_fraud(df, df_fraud, df_fraud2):
  df['fraud'] = np.zeros(len(df))
  frauds_l = df_fraud['invoice_no'].to_list()
  frauds_l2 = df_fraud2['invoice_no'].to_list()
  df.loc[df['invoice_no'].isin(frauds_l),'fraud'] = 1
  df.loc[df['invoice_no'].isin(frauds_l2),'fraud'] = 2
  return df

In [ ]:
df = label_fraud(df, df_fraud,df_fraud2)

In [ ]:
df['fraud'].value_counts()

0.0    228014
2.0       433
1.0       154
Name: fraud, dtype: int64

## Encoding

In [ ]:
df.nunique()

distributor        639
sales              209
branch              28
inv_type             3
invoice_no       30721
product_no        1094
prod_cla             7
qty                202
total_amt        16112
sale_price        8598
ship_qty           202
cust_type           20
return               2
discount_app         2
list_price        6720
inv_ship_days      133
fraud                3
dtype: int64

In [ ]:
cols = df.columns.to_list()
cols = ['distributor', 'sales', 'branch', 'inv_type', 'invoice_no', 'product_no', 'prod_cla', 'cust_type']

In [ ]:
## remove some fields with many classes
cols1 = ['distributor', 'sales','branch', 'invoice_no', 'product_no']
df_less = df.drop(columns=cols1)

In [ ]:
def field_onehotencoding(df):
  cols = [ 'inv_type',  'prod_cla', 'cust_type']
  df[cols] = df[cols].astype('str') 
  df_enc = pd.get_dummies(df)
  return df_enc

In [ ]:
df_less = field_onehotencoding(df_less)

In [ ]:
df_less.shape

(228601, 39)

## Resampling to address the imbalance class
Because fraud labels are given to invoice level, need to keep records of same invoice together. 

In [ ]:
df_fraud = df_less[df_less['fraud'] == 1]
df_non_fraud = df_less[df_less['fraud'] == 0]
df_fraud2 = df_less[df_less['fraud'] == 2]

In [ ]:
df_fraud2.shape

(433, 39)

In [ ]:
df_non_fraud.shape

(228014, 39)

In [ ]:
df_non_fraud.nunique()

qty                   202
total_amt           15896
sale_price           8237
ship_qty              202
return                  2
discount_app            2
list_price           6710
inv_ship_days         128
fraud                   1
inv_type_Z001           2
inv_type_Z016           2
inv_type_Z038           2
prod_cla_11010.0        2
prod_cla_11020.0        2
prod_cla_11030.0        2
prod_cla_11040.0        2
prod_cla_11050.0        2
prod_cla_11060.0        2
prod_cla_9999.0         2
cust_type_AA01          2
cust_type_AA02          2
cust_type_AB01          2
cust_type_AB02          2
cust_type_AB03          2
cust_type_AB04          2
cust_type_AB05          2
cust_type_AB06          2
cust_type_AX01          2
cust_type_CX01          2
cust_type_EX01          2
cust_type_FX01          2
cust_type_GX01          2
cust_type_HX01          2
cust_type_JX01          2
cust_type_KX01          2
cust_type_LX01          2
cust_type_MX01          2
cust_type_NX01          2
cust_type_ZZ

In [ ]:
def sampling_record(df_non_fraud,df_fraud,df_fraud2):
  '''
  Random sampling from unlabled records.
  concat with labeled records
  '''
  df = df_non_fraud.sample(frac=1,replace=False)

  sample_size = len(df_non_fraud) - 8000
 
  ## Build training dataset
  df_train = df.iloc[:sample_size,:]
  df_train = pd.concat([df_train,df_fraud2])
  df_train.loc[df_train['fraud'] == 2, 'fraud'] = 1 
  df_train = df_train.sample(frac=1,replace=False)
  print(len(df_train))

  ## Build eval dataset
  df_test = df.iloc[sample_size:,:]
  df_test = pd.concat([df_test,df_fraud])
  df_test = df_test.sort_index()
  print(len(df_test))

  return df_train, df_test


In [ ]:
df_train, df_test = sampling_record(df_non_fraud, df_fraud, df_fraud2)

220447
8154


In [ ]:
df_train.shape

(220447, 39)

In [ ]:
df_test.shape

(8154, 39)

In [ ]:
df_train.nunique()

qty                   202
total_amt           15729
sale_price           8382
ship_qty              202
return                  2
discount_app            2
list_price           6621
inv_ship_days         132
fraud                   2
inv_type_Z001           2
inv_type_Z016           2
inv_type_Z038           2
prod_cla_11010.0        2
prod_cla_11020.0        2
prod_cla_11030.0        2
prod_cla_11040.0        2
prod_cla_11050.0        2
prod_cla_11060.0        2
prod_cla_9999.0         2
cust_type_AA01          2
cust_type_AA02          2
cust_type_AB01          2
cust_type_AB02          2
cust_type_AB03          2
cust_type_AB04          2
cust_type_AB05          2
cust_type_AB06          2
cust_type_AX01          2
cust_type_CX01          2
cust_type_EX01          2
cust_type_FX01          2
cust_type_GX01          2
cust_type_HX01          2
cust_type_JX01          2
cust_type_KX01          2
cust_type_LX01          2
cust_type_MX01          2
cust_type_NX01          2
cust_type_ZZ

SMOTE 

In [ ]:
y_train = df_train['fraud'].values
X_train = df_train.drop(columns=['fraud'])

In [ ]:
# define pipeline
over = SMOTE(sampling_strategy=0.1)
under = RandomUnderSampler(sampling_strategy=0.5)
steps = [('o', over), ('u', under)]
pipeline = Pipeline(steps=steps)

# transform the dataset
X_train, y_train = pipeline.fit_resample(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
X_train.shape

(66003, 38)

In [ ]:
y_train.sum()

22001.0

In [ ]:
y_train.shape

(66003,)

## Standardscale data

In [ ]:
scaler = StandardScaler()

In [ ]:

X_train = scaler.fit_transform(X_train)
y_train = y_train.reshape(len(y_train),1)

In [ ]:
y_test = df_test['fraud'].values
X_test = df_test.drop(columns=['fraud'])

X_test = scaler.transform(X_test)
y_test = y_test.reshape(len(y_test),1)

In [ ]:
X_train.shape

(66003, 38)

In [ ]:
y_test.shape

(8154, 1)

In [ ]:
y_train.sum()

22001.0

## Build model

In [ ]:
# define the  model
def build_model():
  model = Sequential()
  model.add(Dense(128, input_dim=X_train.shape[1], activation='relu', kernel_initializer='he_normal'))
  model.add(Dense(64,activation='relu'))
  model.add(Dense(32,activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  # compile the keras model
  # opt = tf.keras.optimizers.Adam(clipnorm=1, learning_rate=0.01)
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.Recall()])
  return model

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               8576      
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 16,897
Trainable params: 16,897
Non-trainable params: 0
_________________________________________________________________


## Training model

In [ ]:
## clear previous model.  Make sure training from 0. 
del model
clear_session()
tf.compat.v1.reset_default_graph()
reset_seeds()
model = build_model()

RANDOM SEEDS RESET


In [ ]:
model.fit(X_train, y_train, epochs=1000,batch_size=64,class_weight={0:1, 1:2},validation_data=(X_test,y_test), callbacks=[EarlyStopping(monitor='val_loss', patience=20, mode='min')])

Epoch 1/1000
1032/1032 [==============================] - 4s 4ms/step - loss: 0.0648 - recall: 0.9810 - val_loss: 0.5225 - val_recall: 0.6364
Epoch 2/1000
1032/1032 [==============================] - 2s 2ms/step - loss: 0.0681 - recall: 0.9800 - val_loss: 0.4796 - val_recall: 0.5974
Epoch 3/1000
1032/1032 [==============================] - 2s 2ms/step - loss: 0.0771 - recall: 0.9789 - val_loss: 0.6597 - val_recall: 0.6234
Epoch 4/1000
1032/1032 [==============================] - 2s 2ms/step - loss: 0.0674 - recall: 0.9797 - val_loss: 0.4980 - val_recall: 0.5714
Epoch 5/1000
1032/1032 [==============================] - 2s 2ms/step - loss: 0.0696 - recall: 0.9806 - val_loss: 0.4656 - val_recall: 0.5974
Epoch 6/1000
1032/1032 [==============================] - 2s 2ms/step - loss: 0.0689 - recall: 0.9819 - val_loss: 0.4785 - val_recall: 0.6429
Epoch 7/1000
1032/1032 [==============================] - 2s 2ms/step - loss: 0.0603 - recall: 0.9822 - val_loss: 0.5664 - val_recall: 0.5974
Epoch 

In [ ]:
loss,accuracy = model.evaluate(X_test, y_test)
print(loss, accuracy)

255/255 [==============================] - 1s 1ms/step - loss: 0.6956 - recall: 0.6299
0.6956173181533813 0.6298701167106628


## Predict on eval dataset for further labeling.

In [ ]:
y_pred = model.predict(X_test)
y_cls = (y_pred > 0.5).astype("int32")

In [ ]:
(y_pred > 0.4).sum()

104

In [ ]:
def eval_model(y_test, y_cls):
  y1 = y_test.reshape(-1)
  y2 = y_cls.reshape(-1)
  print(roc_auc_score(y1, y2))
  print(classification_report(y1,y2))
  print(confusion_matrix(y1,y2))


In [ ]:
eval_model(y_train, y_cls)

0.5064665001502765
              precision    recall  f1-score   support

         0.0       0.99      0.67      0.80     36799
         1.0       0.01      0.35      0.02       433

    accuracy                           0.66     37232
   macro avg       0.50      0.51      0.41     37232
weighted avg       0.98      0.66      0.79     37232

[[24527 12272]
 [  283   150]]


In [ ]:
eval_model(y_test, y_cls)

0.763310064935065
              precision    recall  f1-score   support

         0.0       0.99      0.90      0.94      8000
         1.0       0.11      0.63      0.18       154

    accuracy                           0.89      8154
   macro avg       0.55      0.76      0.56      8154
weighted avg       0.98      0.89      0.93      8154

[[7174  826]
 [  57   97]]


In [ ]:
eval_model(y_test, y_cls)

0.763310064935065
              precision    recall  f1-score   support

         0.0       0.99      0.90      0.94      8000
         1.0       0.11      0.63      0.18       154

    accuracy                           0.89      8154
   macro avg       0.55      0.76      0.56      8154
weighted avg       0.98      0.89      0.93      8154

[[7174  826]
 [  57   97]]


In [ ]:
def export_result(df_test,y_cls, y_pred):
  '''
  Attach the predicte result (probability) into original records.
  Export to a csv file for further investication
  '''
  df_test['predict'] = y_cls
  df_test['prob']  = y_pred
  path = '/content/drive/MyDrive/Colab Notebooks/finalproject/'
  file_csv = path + 'pred_result.csv'
  df_test.to_csv(file_csv)
  return True


In [ ]:
export_result(df_test,y_cls,y_pred)

True